# Overview

# Crawl data

In [1]:
#import
import requests
import re
from bs4 import BeautifulSoup
import json
import pandas as pd


In [2]:
#Crawl URL của 3000 sản phẩm và lưu vào file links.txt
def get_id(url):

    # Mở file để ghi
    with open('data/links.txt', 'w') as f:

        # Mỗi trang xe có link của 20 xe
        for i in range(1, 151):
            print(i)
            response = requests.get(url.format(i))

            # Trích xuất phần cần thiết của mã nguồn trang web
            text = response.text

            # Tìm tất cả các cặp "id" và "slug" trong phần đã trích xuất
            matches = re.findall(r'"id":"(\d{8})","slug":"([\w-]+)",', text)

            for match in matches:
                id, slug = match
                # Ghi vào file thay vì in ra
                f.write(f'https://carvago.com/car/{id}/{slug}\n')

# Sử dụng hàm với url đã được chỉnh sửa để chứa số trang
#get_id('https://carvago.com/cars?page={}')

# Crawl thông tin của từng sản phẩm chi tiết

In [3]:
def get_data(text, pattern):
    """
    Trả về tuple chứa các thông tin được trích xuất từ text theo pattern
    """
    match = re.search(pattern, text)
    return match.groups() if match else None

def extract_car_info(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            html = response.content
            soup = BeautifulSoup(html, 'html.parser')
            print('200 OK')

            car_info_divs = soup.find_all('div', class_='css-1b3bvbl')  # Adjust the class if needed

            carname = ""

            for car_info_div in car_info_divs:
                attribute_name_div = car_info_div.find('div', class_='css-f6f0k1')
                if attribute_name_div is not None:
                    attribute_name = attribute_name_div.text.strip()

                    if attribute_name == 'Model':
                        car_link_div = car_info_div.find('div', class_='css-qres8i')
                        if car_link_div is not None:
                            car_link_tag = car_link_div.find('a')
                            if car_link_tag is not None:
                                car_link = car_link_tag['href']
                                carname = car_link.split('/')[2:]  # Extract the last part of the URL
                                carname = " ".join(carname)
            
            # Lấy tên thông tin (ví dụ: 'Make', 'Model', ...)
            keys = []
            keys.append('CARNAME')
            keys.append('ID')
            keys.extend([div.get_text() for div in soup.find_all('div', {'class': 'css-f6f0k1'})])

            # Lấy giá trị thông tin (ví dụ: 'Chevrolet', 'Orlando', ...)
            values = []
            values.append(carname)
            values.append(url.split('/')[4])
            for key, div in zip(keys[2: ], soup.find_all('div', {'class': 'css-1b3bvbl'})):
                if key == 'VIN':
                    if div.find('div', {'class': 'css-10odsig'}) is not None:
                        value = div.find('div', {'class': 'css-10odsig'}).get_text()
                    else:
                        value = div.find('div', {'class': 'css-1yu0dva'}).get_text()
                else:
                    value = div.find('div', {'class': 'css-qres8i'}).get_text()
                values.append(value)

            consumption = [div.get_text() for div in soup.find_all('div', {'class': 'css-c55dw0'})]
            keys.append('Consumption')
            values.append('; '.join(consumption))

            # Lấy giá tiền và đơn vị tiền tệ
            price_pattern = r'"price":(\d+),"price_currency":\{"id":(\d+),"name":"(.*?)","const_key":"CURRENCY_(.*?)"\}'
            price, currency_id, currency_name, currency_const_key = get_data(response.text, price_pattern)
            keys.append('Price')
            values.append(price)
            keys.append('Currency')
            values.append(currency_name)

            # Lấy các thẻ
            tags_pattern = r'"featured_tags":(\[.*?\])'
            featured_tags = json.loads(get_data(response.text, tags_pattern)[0])
            keys.append('Tags')
            #Chuyển các giá trị trong các thẻ thành một string
            values.append('; '.join([tag['name'] for tag in featured_tags]))

            df = pd.DataFrame([values], columns=keys)

            return df
                
    except requests.exceptions.RequestException as e:
        print(f"Failed to get the webpage at {url} due to error: {e}")

In [4]:
# Read links from data.txt
with open('data/links.txt', 'r') as file:
    links = file.read().splitlines()

list_dataframe = []
# Loop through each link and extract information
i = 0
for link in links:
    print(i, '/', len(links))
    car_info = extract_car_info(link)
    i = i+ 1
    if (car_info is None):
        print(f"Skipping link due to error")
        continue  
    list_dataframe.append(car_info)


0 / 3000
Skipping link due to error
1 / 3000
200
2 / 3000
200
3 / 3000
200
4 / 3000
200
5 / 3000
200
6 / 3000
200
7 / 3000


KeyboardInterrupt: 

In [ ]:
crawldata = pd.concat(list_dataframe, ignore_index=True)
crawldata.head(5)

# Lưu vào data/products.csv

In [ ]:
crawldata.to_csv("data/sell_data.csv",header=True)